In [1]:
import itertools
import math as m
import numpy as np
import matplotlib.pyplot as plt

Here we define the Zeta function the same as the previous notebook

$$
Z^{\mathrm{d}}(x^2;\alpha) = \left[\,\sum_{\mathbf{n}\in\mathbb{Z}^3} - \int \! \mathrm{d}^3\mathbf{n} \, \right] \frac{\omega_{n1}^{\star}}{\omega_{n1}} \frac{1}{n^{\star\,2} - x^2} e^{-\alpha(n^{\star\,2} - x^2)}
$$


With the finite volume $F$ function being defined as

$$
F(E,\mathbf{P},L) = \frac{\xi}{8\pi^2 L E^{\star}} \lim_{\alpha \to 0} Z^{\mathrm{P}L/2\pi}\left( (q^{\star}L / 2\pi)^2 ;\alpha \right)
$$


Again, it is convenient to define this function in terms of dimensionless parameters
$$
\bar{E}^{\star} = \frac{E^{\star}L}{2\pi}
$$


$$
F(\bar{E},\mathbf{d},mL) = \frac{\xi}{4\pi\bar{E}^{\star}} \lim_{\alpha \to 0} Z^{\mathrm{P}L/2\pi}\left( x^2 ;\alpha \right)
$$

In [ ]:
'''
 sqrt with branch cut aligned on positive real axis
'''
def mySqrt( z ):
   return 1j * c.sqrt( -z )

'''
 inner product for two list (native)
'''
def dot(A, B):
   return np.dot(A,B) #inner1d(A,B) #np.sum( A * B, axis=1) 

'''
 Function to generate 3-tuple "n = (nx,ny,nz)" list 
 subject to the constraint nx^2 + ny^2 + nz^2 <= nShell^2,
 where "nShell" indicates the number of shells
'''
def generateIntList( nShell ):
    intList = []
    rawList = np.array(np.arange(-nShell,nShell+1))
    for i in itertools.product(rawList,rawList,rawList):
        tmpList = np.array(i)
        if ( np.dot(tmpList,tmpList) <= nShell*nShell ):
            intList.append(tmpList)

    return np.asarray(intList)

#n = [ [a1,a2,a3], [b1,b2,b3], [c1,c2,c3], ... ]

## inner1d(n,n) = [ a.dot(a), b.dot(b), ... ]

## n.dot(unit(beta)) = [ a.dot(unit(beta)), b.dot(unit(beta)), ... ]

'''
 returns the unit vector of a vector "A"
'''
def unit_vector( A ):
    if( all( A==0 ) ):
        return 0.0
    else:
        return A / np.sqrt( np.dot(A,A) )

'''
 special multiply for array of arrays (A) and an array (B), used in the zeta function
'''
def multiply( A, B ):
   nA = len(A)
   return A.reshape(nA,1) * np.tile( B, (nA,1) )
   ### First, reshape A array to 2d (1xnA), and then copy b vector nA time to make (nA x nB) array
   ### alternatively, do (A.reshape(nA,1)).dot( B.reshape(1,3) ) 
   ### bad alternative:  np.full( (nB,nA), A ).T * np.full( (nA,nB) , B )


'''
 returns a Lorentz boosted four vector "(A0,A)" given some boost
 velocity "beta"
'''
def lorentzBoost( A0, A, beta ):
    gamma = 1.0 / np.sqrt( 1.0 - np.dot(beta,beta) )
    Apar  = multiply( A.dot( unit_vector(beta) ), unit_vector(beta) )
    Aper  = A - Apar
    A0_prime = gamma * ( A0 - A.dot( beta ) )
    A_prime  = gamma * ( Apar - multiply( A0, beta ) ) + Aper
    return A0_prime, A_prime

'''
 KSS cutoff function
'''
def cutoff( x_sq, n_sq, alpha ):
    return np.exp( -alpha * ( n_sq - x_sq ) )

'''
 KSS Zeta function for ell = m_ell = 0, given "x_sq", total momentum "nP"
 This function is in units of L / 2pi, e.g. m1 -> m1 * L / (2 * pi).
'''
def zetaFunction( x_sq, nP, alpha ):
    Ecm = np.sqrt( m1_sq + x_sq ) + np.sqrt( m2_sq + x_sq )
    beta = nP / np.sqrt( Ecm * Ecm + np.dot(nP,nP) )    

    n = intList
    n0 = np.sqrt( m1_sq + inner1d(n,n) )
    n0_cm, n_cm = lorentzBoost( n0, n, beta )
    tmp = cutoff( x_sq, inner1d(n_cm,n_cm), alpha ) * n0_cm / n0 
    tmp = tmp / ( inner1d(n_cm,n_cm ) - x_sq ) 
    sum0 = np.sum( tmp )

    eps = 1.0e-16 * ( 1.0 - m.copysign(1, x_sq) )
    x   = c.sqrt( x_sq + 1j * eps )
    z   = c.sqrt( alpha * x_sq )
    integral = 4.0 * m.pi * ( ( m.sqrt( 0.25 * m.pi / alpha ) ) * m.exp( alpha * x_sq )
                              - 0.5 * m.pi * x * s.erfi( z ) )
        
    return sum0 - integral - 4.0 * m.pi * ( 0.5 * 1j * m.pi * x ) # last term needed to compare to BBHO


In [ ]:
def finiteVolumeF( E, nP, L ):
    xi = 1.0
    if ( ident == 'yes' ):
        xi = 0.5
    Ecm = np.sqrt( En**2 - np.dot(nP,nP) )
        Ecm = 2.0 * m.pi * Ecm / mL
    pre_factor = xi / ( 4.0 * 2.0 * m.pi * mL * Ecm_o_m )
    return xi  

In [ ]:
alpha = .1 #( 1.0 / 3.0 )**4
nShell = 20

intList = generateIntList( nShell )